<a href="https://colab.research.google.com/github/soheilpaper/-tft-2.4-ili9341-STM32/blob/master/RAG's/RAG-from-Scratch/Llama3_RAG_for_web_url_2_sumerized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
# Uninstall old versions (optional)
!pip uninstall -y faiss faiss-cpu faiss-gpu

!pip install --upgrade gradio #==3.13.0
!pip install langchain langchain-ollama faiss-cpu beautifulsoup4 ollama
!pip install langchain-community unstructured openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 35.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.59
    Uninstalling langchain-core-0.3.59:
      Successfully uninstalled langchain-core-0.3.59
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.

In [ ]:
#!wget 'http://213.188.192.134/files/iran-oil_iran_oil.xlsx'
!wget 'https://huggingface.co/spaces/sosa123454321/Exhibition-connector-rag1/resolve/main/iran-oil_iran-oil_iran%20oil.xlsx'

--2025-05-26 12:38:32--  https://huggingface.co/spaces/sosa123454321/Exhibition-connector-rag1/resolve/main/iran-oil_iran-oil_iran%20oil.xlsx
Resolving huggingface.co (huggingface.co)... 18.160.143.75, 18.160.143.99, 18.160.143.32, ...
Connecting to huggingface.co (huggingface.co)|18.160.143.75|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/03/b6/03b6b42a86fc6f3407b6a16887f21398fac4c268f3ab10c184ee04296bc7f67d/7f7e69635fcac98bcefa244620f29990a52dae4b583ce7a1eb627434e168ce5e?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27iran-oil_iran-oil_iran%252520oil.xlsx%3B+filename%3D%22iran-oil_iran-oil_iran%2520oil.xlsx%22%3B&response-content-type=application%2Fvnd.openxmlformats-officedocument.spreadsheetml.sheet&Expires=1748266712&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0ODI2NjcxMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8wMy9iNi8wM2I2YjQyYTg2ZmM2ZjM0MDdiNm

In [4]:
# type these codes in below terminal after run the cell (%xterm)
! curl -fsSL https://ollama.com/install.sh | sh
#! ollama serve & ollama pull llama3 & ollama pull nomic-embed-text

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
# Start Ollama server in background with nohup
!nohup ollama serve > ollama.log 2>&1 &

# Pull models (only need to do once)
!ollama pull llama3
!ollama pull nomic-embed-text

In [ ]:
!pip install ollama langchain beautifulsoup4 chromadb gradio

In [5]:
import os
import time
import gradio as gr
import ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

# Wait for Ollama server to start (adjust if needed)
print("Waiting for Ollama server to start...")
time.sleep(1)

# Set USER_AGENT to avoid scraping warnings
os.environ["USER_AGENT"] = "MyRAGApp/1.0 (+https://example.com)"

try:
    # Load Wikipedia page documents
    wiki_url = 'https://en.wikipedia.org/wiki/Ohiya'
    print(f"Loading Wikipedia documents from {wiki_url} ...")
    wiki_loader = WebBaseLoader(wiki_url)
    wiki_docs = wiki_loader.load()
    print(f"Loaded {len(wiki_docs)} Wikipedia documents.")

    # Load Excel file documents
    excel_url = 'https://huggingface.co/spaces/sosa123454321/Exhibition-connector-rag1/resolve/main/iran-oil_iran-oil_iran%20oil.xlsx'
    print(f"Loading Excel documents from {excel_url} ...")
    excel_loader = UnstructuredExcelLoader(excel_url, mode="elements")
    excel_docs = excel_loader.load()
    print(f"Loaded {len(excel_docs)} Excel documents.")

    # Combine documents from both sources
    all_docs = wiki_docs + excel_docs
    print(f"Total documents combined: {len(all_docs)}")

    # Split documents into chunks for embedding
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(all_docs)
    print(f"Split into {len(splits)} document chunks.")

    # Create embeddings and vector store
    print("Checking Ollama embedding model availability...")
    # You may want to check Ollama models here or pull if needed externally

    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    print("Creating FAISS vector store from document chunks...")
    vectorstore = FAISS.from_documents(splits, embeddings)
    print("Vector store created successfully.")

    # Retriever from vector store
    retriever = vectorstore.as_retriever()

except Exception as e:
    print(f"Error during setup: {e}")
    # If setup fails, we still want the app to launch but inform user
    wiki_docs = []
    excel_docs = []
    all_docs = []
    splits = []
    retriever = None

# Define Ollama Llama3 query function with error handling
def ollama_llm(question, context):
    prompt = f"Question: {question}\n\nContext: {context}"
    try:
        response = ollama.chat(model='llama3', messages=[{'role': 'user', 'content': prompt}])
        return response['message']['content']
    except Exception as e:
        print(f"Ollama LLM error: {e}")
        return f"Error in LLM response: {e}"

# RAG chain function with error handling
def rag_chain(question):
    if retriever is None:
        return "Retriever not initialized due to setup error."
    try:
        docs = retriever.get_relevant_documents(question)
        if not docs:
            return "No relevant documents found for your query."
        context = "\n\n".join(doc.page_content for doc in docs)
        return ollama_llm(question, context)
    except Exception as e:
        print(f"Error in rag_chain: {e}")
        return f"Error during retrieval or LLM call: {e}"

# Gradio interface function with error handling
def get_important_facts(question):
    try:
        if not question.strip():
            return "Please enter a valid question."
        return rag_chain(question)
    except Exception as e:
        print(f"Error in get_important_facts: {e}")
        return f"Unexpected error: {e}"

# Create Gradio interface
iface = gr.Interface(
    fn=get_important_facts,
    inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
    outputs="text",
    title="RAG with Llama3 and Excel Data",
    description="Ask questions about the Wikipedia page on Ohiya and the Iran oil Excel data.",
)

# Launch Gradio app with debug=True to see logs and share=True for public link
if __name__ == "__main__":
    iface.launch(share=True, debug=True)

Waiting for Ollama server to start...
Loading Wikipedia documents from https://en.wikipedia.org/wiki/Ohiya ...
Loaded 1 Wikipedia documents.
Loading Excel documents from https://huggingface.co/spaces/sosa123454321/Exhibition-connector-rag1/resolve/main/iran-oil_iran-oil_iran%20oil.xlsx ...
Loaded 1 Excel documents.
Total documents combined: 2
Split into 317 document chunks.
Checking Ollama embedding model availability...
Creating FAISS vector store from document chunks...
Error during setup: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://10e8115171bec47216.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (http

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://10e8115171bec47216.gradio.live


In [ ]:
import os
import time
import gradio as gr
import ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
import subprocess
import re
import json
from datetime import datetime
import base64
import uuid

# Ensure gTTS is installed and up-to-date
try:
    from gtts import gTTS
except ImportError:
    print("Installing gTTS...")
    !pip install gTTS --upgrade
    from gtts import gTTS

# Check gTTS version for debugging
import gtts
print(f"gTTS version: {gtts.__version__}")

# Pull the required models
!ollama pull llama3
!ollama pull nomic-embed-text

# Start the Ollama server in the background
print("Starting Ollama server...")
subprocess.Popen(["ollama", "serve"])
time.sleep(15)  # Wait 15 seconds to ensure server is up

# Check if Ollama server is running
try:
    ollama.list()  # Test connection
    print("Ollama server is running.")
except Exception as e:
    print(f"Failed to connect to Ollama server: {e}")
    print("Please ensure Ollama is installed and running.")
    exit(1)

# Set USER_AGENT to avoid scraping warnings
os.environ["USER_AGENT"] = "MyRAGApp/1.0 (+https://example.com)"

# Load Wikipedia page documents
wiki_url = 'https://en.wikipedia.org/wiki/Ohiya'
wiki_loader = WebBaseLoader(wiki_url)
wiki_docs = wiki_loader.load()

# Load Excel file documents
excel_url = 'https://huggingface.co/spaces/sosa123454321/Exhibition-connector-rag1/resolve/main/iran-oil_iran-oil_iran%20oil.xlsx'
excel_loader = UnstructuredExcelLoader(excel_url, mode="elements")
try:
    excel_docs = excel_loader.load()
    print(f"Loaded {len(excel_docs)} Excel documents")
except Exception as e:
    print(f"Excel loading error: {e}")
    excel_docs = []

# Combine documents from both sources
all_docs = wiki_docs + excel_docs

# Split documents into chunks for embedding
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(all_docs)

# Create embeddings and vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = FAISS.from_documents(splits, embeddings)

# Define Ollama Llama3 query function with Persian language preference
def ollama_llm(question, context):
    system_message = "You are a helpful assistant. Please respond in Persian (Farsi)."
    prompt = f"{system_message}\n\nQuestion: {question}\n\nContext: {context}"  # Fixed f-string
    print(f"Generated prompt: {prompt[:100]}...")  # Debug print
    response = ollama.chat(model='llama3', messages=[
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': prompt}
    ])
    return response['message']['content']

# Retriever from vector store
retriever = vectorstore.as_retriever()

# RAG chain function
def rag_chain(question):
    docs = retriever.invoke(question)
    context = "\n\n".join(doc.page_content for doc in docs)
    return ollama_llm(question, context)

# Chat history management
chat_history_file = "chat_history.json"

def load_chat_history():
    if os.path.exists(chat_history_file):
        try:
            with open(chat_history_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        except:
            return []
    return []

def save_chat_history(history):
    with open(chat_history_file, 'w', encoding='utf-8') as f:
        json.dump(history, f, ensure_ascii=False, indent=2)

# Text-to-speech function with fallback
def text_to_speech(text):
    try:
        print("Generating TTS for text:", text[:50], "...")
        tts = gTTS(text=text, lang='fa')  # Persian language
        audio_file = f"tts_{uuid.uuid4().hex}.mp3"
        tts.save(audio_file)
        with open(audio_file, "rb") as f:
            audio_data = f.read()
            audio_base64 = base64.b64encode(audio_data).decode('utf-8')
        os.remove(audio_file)
        print("TTS generated successfully.")
        return f'<audio controls><source src="data:audio/mp3;base64,{audio_base64}" type="audio/mp3"></audio>'
    except Exception as e:
        print(f"TTS error: {str(e)}")
        return "خطا در تولید صدا: در حال حاضر تولید صدا برای فارسی پشتیبانی نمی‌شود. لطفاً متن پاسخ را بخوانید."

# Format response with hyperlinks and images
def format_response(response):
    image_pattern = r'(https?://[^\s]+\.(jpg|png|gif|bmp))'
    response = re.sub(image_pattern, r'<img src="\1" alt="Image" width="300"/>', response)
    url_pattern = r'(https?://[^\s]+)'
    response = re.sub(url_pattern, r'<a href="\1" target="_blank">\1</a>', response)
    return response

# Main chatbot function
def chatbot_function(question):
    if not question.strip():
        return "لطفاً یک سوال معتبر وارد کنید.", None

    persistent_history = load_chat_history()
    response = rag_chain(question)
    formatted_response = format_response(response)
    audio_html = text_to_speech(response)
    timestamp = datetime.now().isoformat()
    persistent_history.append({
        "user": question,
        "bot": formatted_response,
        "timestamp": timestamp
    })
    save_chat_history(persistent_history)
    history_display = ""
    for entry in persistent_history:
        history_display += f"<div style='direction: rtl; margin: 10px;'><strong>کاربر:</strong> {entry['user']}<br><strong>ربات:</strong> {entry['bot']}</div>"
    return f"{history_display}<hr>{formatted_response}", audio_html

# Clear chat history function
def clear_history():
    if os.path.exists(chat_history_file):
        os.remove(chat_history_file)
    return "تاریخچه گفتگو پاک شد."

# Custom CSS for RTL and user-friendly design
custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Vazirmatn:wght@400;700&display=swap');
body {
    font-family: 'Vazirmatn', sans-serif;
    background-color: #f0f4f8;
    direction: rtl;
    text-align: right;
}
.gradio-container {
    max-width: 800px;
    margin: auto;
    padding: 20px;
}
input[type=text], textarea {
    direction: rtl;
    border: 2px solid #007bff;
    border-radius: 5px;
    width: 100%;
    padding: 10px;
}
button {
    background-color: #007bff;
    color: white;
    border: none;
    padding: 10px 20px;
    border-radius: 5px;
    cursor: pointer;
}
button:hover {
    background-color: #0056b3;
}
.output-text {
    background-color: #d1e7dd;
    border-radius: 10px;
    padding: 10px;
    margin: 10px 0;
}
"""

# Create Gradio interface
with gr.Blocks(css=custom_css) as iface:
    gr.Markdown("<h1 style='text-align: center;'>چت‌بات RAG با Llama3</h1>")
    gr.Markdown("<p style='text-align: center;'>سوال خود را در مورد داده‌های نفت ایران یا اطلاعات دیگر بپرسید.</p>")

    question_input = gr.Textbox(placeholder="سوال خود را اینجا وارد کنید...", label="سوال شما")
    output_text = gr.HTML(label="پاسخ")
    output_audio = gr.HTML(label="صدا")

    submit_btn = gr.Button("ارسال")
    clear_btn = gr.Button("پاک کردن تاریخچه")

    submit_btn.click(
        fn=chatbot_function,
        inputs=question_input,
        outputs=[output_text, output_audio]
    )
    clear_btn.click(
        fn=clear_history,
        inputs=None,
        outputs=output_text
    )

# Launch Gradio app with share=True for Colab
if __name__ == "__main__":
    iface.launch(share=True, debug=True)